In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests as req
import pickle
import re
import pandas as pd
import numpy as np
from io import BytesIO
from PIL import Image
import functools
import operator

In [2]:
# define a function which scrape url based on number of pages

# subfunction 1
def get_carModelUrl(url):
    html = req.get(url)
    soup = BeautifulSoup(html.content, 'lxml')
    href = soup.body.find('div', {'id':'content'}).find('form', {'name':'searchform'}).next_sibling.next_sibling.find_all(href=re.compile(r"(^(info.php).*)"), string=True)
    return href

# subfunction 2
def get_keypairs(hrefString):
    
    idValue = re.search("(?<=\?ID=)\d+(?=\&)", str(hrefString))
    dlValue = re.search("(?<=\;DL=)\d+(?=(\"|\&))", str(hrefString))
    
    return idValue[0].strip(), dlValue[0].strip()

# main function 1: get the webpage url
def scrape_url(page):
    
    #convert page to list of car per page
    pageToList = [(i+1)*100 for i in range(page)]
    
    #create BeautifulSoup object
    urlList = [f"https://www.sgcarmart.com/used_cars/listing.php?BRSR={car}&RPG=100" for car in pageToList]
    carModelUrlList = list(map(get_carModelUrl, urlList))
    
    #flatten the list
    carModelUrlList = functools.reduce(operator.iconcat, carModelUrlList, [])
    
    #get keypairs(id-dl)
    keypairsList = list(map(get_keypairs, carModelUrlList))
    
    #reformat the webpage url
    carModelUrlList = list(map(lambda x: f"https://www.sgcarmart.com/used_cars/info.php?ID={x[0]}&DL={x[1]}", keypairsList))
    
    return carModelUrlList

In [3]:
# main function 2: get the image link
def scrape_image(url):
    html = req.get(url)
    soup = BeautifulSoup(html.content, 'lxml')
    
    image_url = ''
    try:
        image_url = soup.body.find('div', {'id':'contentblank'}).select('div:nth-of-type(2)')[0].select('div:nth-of-type(5)')[0].select('div:nth-of-type(1)')[0].next_sibling.next_sibling.findAll('img')[0].attrs['src']
    except:
        image_url = None
        pass
    
    model = soup.body.find('div', {'id':'contentblank'}).select('div:nth-of-type(2)')[0].select('div:nth-of-type(1)')[0].text.strip()
    #clean the model text
    if model.find("(") >= 0:
        model = model.partition("(")[0]
    
    return model, image_url

In [13]:
# subfunction 1
def download_image(model, url):
    # use pillow to read the btye string
    response = req.get(url)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        #specify your directory below
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(f"Y:/Documents/GitHub/CarSmartConsultancy/Data/img_data/{model}.jpg")

# main function 3
def save_image(df):
    return np.where(df['Image'] != None, download_image(df['Model'], df['Image']), None)

In [11]:
# run main function 1: get webpage url based o
carModelUrls = scrape_url(1)
%time images = list(map(scrape_image, carModelUrls))

CPU times: total: 28.6 s
Wall time: 57.1 s


In [6]:
# convert the model and image link to a dataframe
df = pd.DataFrame(images, columns=['Model', 'Image'])

In [14]:
# run main function 3: download image to directory
import swifter
df.swifter.apply(save_image, axis=1)

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\lkimhui\anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object